## 課題提出時（2018/11/28）の所感
最初、MNIST対応を考えていましたが、サンプルデータだけに対応することを目指して、一次元の畳み込みを体感できれば良いとのことでしたのでその対応を行いました。一部ハードコードしてしまっている箇所がありますが、np.array（L78,80）でインデックスの指定方法があることを学べましたのでよかったです。一旦こちらで提出します。

オプションは対応しておりません。

# 1.この課題の目的
・スクラッチを通してCNNの基礎を理解する  

【目的としないこと】  
・実用的なCNNモデルに関する知識の習得

# 2.進め方
・1次元畳み込み層をスクラッチで実装する。（1日目〜2日目）  
sprint13も引き続きCNNを扱う。sprint12では1次元畳み込み層をスクラッチで作成し、畳み込みの基礎を理解することを目指す。sprint13で2次元畳み込み層とプーリング層を作成し、CNNの全体を理解する。説明課題もsprint13で行う

# 3.スクラッチによる実装
NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していく。TensorFlowなどは使用しない。
sprint10で作成したディープニューラルネットワークを拡張する形で作成する。

## 今回確認するPythonのスキル
・ndarrayの配列を使用したインデックス指定方法

## 1次元畳み込み層
画像に対する畳み込みは通常2次元畳み込みを行うが、sprint12ではまず1次元畳み込みを実装する。段階的に複雑なものに挑戦していくことにする。

なお、通常1次元畳み込みは系列データで使われることが多い。畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークに一般的に用意されている。

## データセットの用意
画像に対する畳み込みは通常2次元畳み込みを行うが、sprint12ではまず1次元畳み込みを実装する。段階的に複雑なものに挑戦していくことにする。

なお、通常1次元畳み込みは系列データで使われることが多い。畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークに一般的に用意されている。

In [52]:
import numpy as np
x = np.array([1,2,3,4])
w = np.array([3,5,7])
b = np.array([1])

loss = np.array([10, 20])
y = np.array([25, 30])

In [53]:
import numpy as np

# sprint10のFCクラスに相当
class SimpleConv1d:
    
    def __init__(self):
        # 予測値
        self.y_pred = []
        
        # 勾配
        self.dB = None # バイアスの勾配
        self.dW = None # 重みの勾配
        self.delta = None # 受け取った勾配
        self.dx = None # 後ろに流す勾配
        
        # 
        self.P = 0  # パディング数
        self.S = 1 # ストライド数
        self.F = None # フィルタ数
        self.N_in = None # 入力のサイズ（特徴量の数）
        self.N_out = None # 出力のサイズ（特徴量の数）
        
        # 入力値・重み
        self.x = None
        self.w = None
        
    def forward(self,x, w, b):
        
        self.x = x
        self.w = w

        # 入力サイズ（特徴量の数）・・・今回は4になる
        self.N_in = x.shape[0]
        
        # フィルターサイズ＝重みのサイズ・・・今回は3になる
        self.F = self.w.shape[0]

        # 1次元畳み込み後の出力サイズ　・・・今回は２になる
        self.N_out = ((self.N_in + 2*self.P - self.F) / self.S) + 1
        self.N_out = int(self.N_out)
        
        for i in range(int(self.N_out)): # i  :  0~2(F=3の場合)
            self.y_pred.append(np.dot(x[i : self.F+i], self.w.T) + b)
        
        # 予測値
        self.y_pred = np.array(self.y_pred)
        
    
    def backward(self, delta):
        """
        dA : 後ろから流れてきた勾配
        dZ : 前に流す勾配 
        """     
        self.dW = np.zeros(self.F)
        self.dx = []
        
        # バイアスは加算して流す
        self.dB = np.sum(delta)
        self.dB = np.array([self.dB])
        
        # 重みパラメータの算出（フィルタのサイズでループを回す）　self.N_out = 2
        for i in range(self.N_out):
            self.dW[i] = np.dot(delta, self.x[i : self.N_out+i])
            
        # self.N_in = 4  入力の特徴量の数
        for i in range(self.N_in):
            # 最初の行の処理
            if i == 0:
                self.dx.append(delta[i]*self.w[i])                
            
            # 最後の行の処理
            elif i == self.N_in-1:
                self.dx.append(delta[-1]*self.w[i-1])
                
            # 真ん中の行の処理
            else:
                # ハードコードになってしまっている。。
                indexces = np.array([i,i-1])
                #indexces = np.arange(i,i-1])
                self.dx.append(np.dot(delta.T, self.w[indexces]))
        
        self.dx = np.array(self.dx, dtype=int)
        self.dx = self.dx.T.flatten()
        self.dW = np.array(self.dW, dtype=int)
        
        return self.dB, self.dW,self.dx

In [54]:
sc = SimpleConv1d()
sc.forward(x, w, b)
sc.backward(loss)

(array([30]), array([50, 80,  0]), array([ 30, 110, 170, 140]))

# （オプション）実装上のさらなる工夫
タイムオーバーで出来ておりません

In [ ]:
# shape(2,4)で、（入力チャンネル数、特徴量数）である。
x_option = np.array([[1,2,3,4],[2,3,4,5]])

# 例を簡略化するため全て1とする。
# （出力チャンネル数、入力チャンネル数、フィルタサイズ）である。
w_option = np.ones((3,2,3))

# （出力チャンネル数）
b_option = np.array([1,2,3])

# 出力は次のようになる
# shape(3,2)で（出力チャンネル数、特徴量数）である。
a = np.array([[16,22],[17,23],[18,24]])
